# 2. Grouping customers together!

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
data = pd.read_csv("bank_transactions.csv")

In [76]:
data.TransactionDate  = pd.to_datetime(data.TransactionDate)

In [80]:
def convert_time(value):
    sec = str(value % 100)
    minutes = str((value //100) %100)
    hours = str((value // 10000) %100)
    return datetime.strptime(str(hours)+":"+str(minutes)+":"+str(sec), '%H:%M:%S').time()

In [ ]:
data.TransactionTime = data.TransactionTime.apply(lambda row: convert_time(row))

In [119]:
data.TransactionDate = pd.to_datetime(data.TransactionDate)

In [120]:
data

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),difference,n_significant_transactions,n_normal_transactions,early,late
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2016-02-08,14:32:07,25.0,17794.05,0,1,0,1
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2016-02-08,14:18:58,27999.0,-25728.31,1,0,0,1
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2016-02-08,14:27:12,459.0,17415.44,0,0,0,1
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2016-02-08,14:27:14,2060.0,864443.21,0,1,0,1
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2016-02-08,18:11:56,1762.5,4951.93,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,8/4/90,M,NEW DELHI,7635.19,2016-09-18,18:48:24,799.0,6836.19,1,0,0,1
1048563,T1048564,C6459278,20/2/92,M,NASHIK,27311.42,2016-09-18,18:37:34,460.0,26851.42,0,0,0,1
1048564,T1048565,C6412354,18/5/89,M,HYDERABAD,221757.06,2016-09-18,18:33:13,770.0,220987.06,0,1,0,1
1048565,T1048566,C6420483,30/8/78,M,VISAKHAPATNAM,10117.87,2016-09-18,18:47:06,1000.0,9117.87,0,0,0,1


The 7 features recommended in the homework are the following:

a) Number of transactions: *n_transactions*

b) Number of transactions with a balance bigger than 100 USD: *n_transactions_100*

c) Average amount of the transactions: *average_amount*

d) Average balance: *average_balance*

e) Average difference between the balance and the transaction amount for each customer (this is mainly known in the banking world as utilisation): *utilisation*

f) Most common gender of the customer: *common_gender*

h) Most frequent location of the customer: *frequent_location*

### 2.1 Getting your data + feature engineering

In [4]:
transactions = data.groupby(['CustomerID'])['CustomerID'].count().reset_index(name='n_transactions')

In [5]:
trans_100 = data.groupby(['CustomerID'])["TransactionAmount (INR)"].apply(lambda x: (x>100).sum()).reset_index(name='n_transactions_100')

In [6]:
features = pd.merge(transactions,trans_100, on="CustomerID")

In [7]:
amount= data.groupby(['CustomerID'])["TransactionAmount (INR)"].mean().reset_index(name='average_amount')

In [8]:
features = pd.merge(features,amount, on="CustomerID")

In [9]:
av_balance= data.groupby(['CustomerID'])["CustAccountBalance"].mean().reset_index(name='average_balance')

In [10]:
features = pd.merge(features, av_balance, on="CustomerID")

In [11]:
data['difference'] = data["CustAccountBalance"] - data["TransactionAmount (INR)"]

In [12]:
utilisation = data.groupby(["CustomerID"])["difference"].mean().reset_index(name='utilisation')

In [13]:
features = pd.merge(features, utilisation, on="CustomerID")

In [14]:
gender= data.groupby(['CustomerID'])['CustGender'].agg(pd.Series.mode).reset_index(name='common_gender')

In [15]:
features = pd.merge(features,gender, on="CustomerID")

In [16]:
freq_location= data.groupby(['CustomerID'])['CustLocation'].agg(pd.Series.mode).reset_index(name='frequent_location')

In [17]:
features = pd.merge(features,freq_location, on="CustomerID")

In [18]:
features

,CustomerID,n_transactions,n_transactions_100,average_amount,average_balance,utilisation,common_gender,frequent_location
0,C1010011,2,2,2553.0,76340.635,73787.635,"[F, M]","[NEW DELHI, NOIDA]"
1,C1010012,1,1,1499.0,24204.490,22705.490,M,MUMBAI
2,C1010014,2,2,727.5,100112.950,99385.450,"[F, M]",MUMBAI
3,C1010018,1,0,30.0,496.180,466.180,F,CHAMPARAN
4,C1010024,1,1,5000.0,87058.650,82058.650,M,KOLKATA
...,...,...,...,...,...,...,...,...
884260,C9099836,1,1,691.0,133067.230,132376.230,M,BHIWANDI
884261,C9099877,1,1,222.0,96063.460,95841.460,M,BANGALORE
884262,C9099919,1,1,126.0,5559.750,5433.750,M,GUNTUR
884263,C9099941,1,0,50.0,35295.920,35245.920,M,CHENNAI


## Consider at least 20 additional features that can be generated for each CustomerId. Describe each of them and why you believe they will be helpful. Moreover, add it to the previous dataset (the one with seven features). In the end, you should have for each CustomerID at least 27 features (7 recommended + 20 suggested by you).

The following features are 20 additional features that can be generated for each CustomerId:

1) The minimum TransactionAmount (INR): *min_trans*

2) The maximum TransactionAmount (INR): *max_trans*

3) The minimum CustAccountBalance: *min_accountbalance*

4) The maximum CustAccountBalance: *max_accountbalance*

5) Number of CustAccountBalance with an overall higher than 50000 USD: *account_balance_sup50k*

6) Number of CustAccountBalance with an overall lower than 10000 USD: *account_balance_low10k*

7) The earlier TransactionTime: *earlier_TransactionTime*

8) The latest TransactionTime: *latest_TransactionTime*

9) Amount of the average amount of the transaction on the average account balance: *percentage_of_expenditure*

10) Number of transactions (per person) with a percentage of expenditure greater or equal to 10%: *n_significant_transactions*

11) Number of transaction (per person) with a percentage of expenditure lower or equal to 1%: *n_normal_transactions*

12) Account balance range: *acc_bal_range*

13) Utilisation on average account balance: *percent_availability*

14) Number of transactions in the first half of the day

15) Number of transactions in the second half of the day

16) Number of transactions in the first half of the year

17) Number of transactions in the second half of the year

18) TransactionAmount (INR) median

19) TransactionAmount (INR) first quartile

20) TransactionAmount (INR) third quartile

### Minimum and maximum TransactionAmount (INR)

In [19]:
min_trans= data.groupby(['CustomerID'])["TransactionAmount (INR)"].min().reset_index(name='min_trans')

In [20]:
features = pd.merge(features,min_trans, on="CustomerID")

In [21]:
max_trans= data.groupby(['CustomerID'])["TransactionAmount (INR)"].max().reset_index(name='max_trans')

In [22]:
features = pd.merge(features, max_trans, on="CustomerID")

### Minimum and Maximum CustAccountBalance

In [23]:
min_account= data.groupby(['CustomerID'])["CustAccountBalance"].min().reset_index(name='min_accountbalance')

In [24]:
features = pd.merge(features, min_account, on="CustomerID")

In [25]:
max_account= data.groupby(['CustomerID'])["CustAccountBalance"].max().reset_index(name='max_accountbalance')

In [26]:
features = pd.merge(features, max_account, on="CustomerID")

### Number of CustAccountBalance with an overall higher than 50000 USD: *account_balance_sup50k*

In [27]:
account_balance_sup500k = data.groupby(['CustomerID'])["CustAccountBalance"].apply(lambda x: (x>50000).sum()).reset_index(name='account_balance_sup50k')

In [28]:
features = pd.merge(features, account_balance_sup500k, on="CustomerID")

### Number of CustAccountBalance with an overall lower than 10000 USD: *account_balance_low10k*

In [29]:
account_balance_low10k = data.groupby(['CustomerID'])["CustAccountBalance"].apply(lambda x: (x<10000).sum()).reset_index(name='account_balance_low10k')

In [30]:
features = pd.merge(features, account_balance_low10k, on="CustomerID")

### Earliest and latest TransactionTime

In [87]:
earliest_TransactionTime= data.groupby(['CustomerID'])["TransactionTime"].min().reset_index(name='earliest_TransactionTime')

In [88]:
features = pd.merge(features, earliest_TransactionTime, on="CustomerID")

In [89]:
latest_TransactionTime= data.groupby(['CustomerID'])["TransactionTime"].max().reset_index(name='latest_TransactionTime')

In [90]:
features = pd.merge(features, latest_TransactionTime, on="CustomerID")

### Amount of the average amount of the transaction on the average account balance: *percentage_of_expenditure*

In [37]:
features['percentage_of_expenditure']=round((features['average_amount']/features['average_balance'])*100, 2)

In [38]:
features

,CustomerID,n_transactions,n_transactions_100,average_amount,average_balance,utilisation,common_gender,frequent_location,min_trans,max_trans,min_accountbalance,max_accountbalance,account_balance_sup50k,account_balance_low10k,earliest_TransactionTime,latest_TransactionTime,percentage_of_expenditure
0,C1010011,2,2,2553.0,76340.635,73787.635,"[F, M]","[NEW DELHI, NOIDA]",356.0,4750.0,32500.73,120180.54,1,0,11229,123813,3.34
1,C1010012,1,1,1499.0,24204.490,22705.490,M,MUMBAI,1499.0,1499.0,24204.49,24204.49,0,0,204409,204409,6.19
2,C1010014,2,2,727.5,100112.950,99385.450,"[F, M]",MUMBAI,250.0,1205.0,38377.14,161848.76,1,0,154451,220305,0.73
3,C1010018,1,0,30.0,496.180,466.180,F,CHAMPARAN,30.0,30.0,496.18,496.18,0,1,170254,170254,6.05
4,C1010024,1,1,5000.0,87058.650,82058.650,M,KOLKATA,5000.0,5000.0,87058.65,87058.65,1,0,141103,141103,5.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884260,C9099836,1,1,691.0,133067.230,132376.230,M,BHIWANDI,691.0,691.0,133067.23,133067.23,1,0,5122,5122,0.52
884261,C9099877,1,1,222.0,96063.460,95841.460,M,BANGALORE,222.0,222.0,96063.46,96063.46,1,0,120255,120255,0.23
884262,C9099919,1,1,126.0,5559.750,5433.750,M,GUNTUR,126.0,126.0,5559.75,5559.75,0,1,122533,122533,2.27
884263,C9099941,1,0,50.0,35295.920,35245.920,M,CHENNAI,50.0,50.0,35295.92,35295.92,0,0,213722,213722,0.14


### Number of transactions (per person) with a percentage of expenditure greater or equal to 10%: n_significant_transactions

In [40]:
TransactionAmount=data['TransactionAmount (INR)']

In [41]:
CustAccountBalance=data['CustAccountBalance']

In [42]:
data['n_significant_transactions']=0

In [43]:
for i in range(len(data)):
    if TransactionAmount[i]>=CustAccountBalance[i]*0.1:
        data['n_significant_transactions'][i]=1

C:\Users\feder\AppData\Local\Temp\ipykernel_19568\2877739962.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['n_significant_transactions'][i]=1


In [44]:
n_significant_transactions=data.groupby(['CustomerID'])['n_significant_transactions'].sum().reset_index(name='n_significant_transactions')

In [45]:
features = pd.merge(features, n_significant_transactions, on="CustomerID")

### Number of transaction (per person) with a percentage of expenditure lower or equal to 1%: *n_normal_transactions*

In [49]:
TransactionAmount=data['TransactionAmount (INR)']

In [50]:
CustAccountBalance=data['CustAccountBalance']

In [51]:
data['n_normal_transactions']=0

In [53]:
for i in range(len(data)):
    if TransactionAmount[i]<=CustAccountBalance[i]*0.01:
        data['n_normal_transactions'][i]=1

C:\Users\feder\AppData\Local\Temp\ipykernel_19568\856099615.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['n_normal_transactions'][i]=1


In [54]:
n_normal_transactions=data.groupby(['CustomerID'])['n_normal_transactions'].sum().reset_index(name='n_normal_transactions')

In [55]:
features = pd.merge(features, n_normal_transactions, on="CustomerID")

###  Account balance range: *acc_bal_range*

In [59]:
features['acc_bal_range']=features['max_accountbalance']-features['min_accountbalance']

### Utilisation on average account balance: *percent_availability*

In [83]:
features['percent_availability']=round(features['utilisation']/features['average_balance']*100, 2)

### Number of transactions in the first half of the day

In [105]:
transaction_time=data['TransactionTime']

In [106]:
data['early']=0

In [107]:
for i in range(len(data)):
    if transaction_time[i]<=datetime.strptime("12:00:00", '%H:%M:%S').time():
        data['early'][i]=1

C:\Users\feder\AppData\Local\Temp\ipykernel_19568\2474452796.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['early'][i]=1


In [111]:
first_half_transactions=data.groupby(['CustomerID'])['early'].sum().reset_index(name='first_half_transactions')

In [112]:
features = pd.merge(features, first_half_transactions, on="CustomerID")

###  Number of transactions in the second half of the day

In [114]:
data['late']=0

In [115]:
for i in range(len(data)):
    if transaction_time[i]>datetime.strptime("12:00:00", '%H:%M:%S').time():
        data['late'][i]=1

C:\Users\feder\AppData\Local\Temp\ipykernel_19568\496431693.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['late'][i]=1


In [116]:
second_half_transactions=data.groupby(['CustomerID'])['late'].sum().reset_index(name='second_half_transactions')

In [117]:
features = pd.merge(features, second_half_transactions, on="CustomerID")

### Number of transactions in the first half of the year

In [121]:
data['first_half']=0

In [122]:
for i in range(len(data)):
    a=str(data.TransactionDate[i])
    b=int(a[5]+a[6])
    if b<=6:
        data['first_half'][i]=1

C:\Users\feder\AppData\Local\Temp\ipykernel_19568\2107607953.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['first_half'][i]=1


In [124]:
first_months_transactions=data.groupby(['CustomerID'])['first_half'].sum().reset_index(name='first_months_transactions')

In [125]:
features = pd.merge(features, first_months_transactions, on="CustomerID")

### Number of transactions in the second half of the year

In [127]:
data['second_half']=0

In [128]:
for i in range(len(data)):
    a=str(data.TransactionDate[i])
    b=int(a[5]+a[6])
    if b>6:
        data['second_half'][i]=1

C:\Users\feder\AppData\Local\Temp\ipykernel_19568\3048449531.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['second_half'][i]=1


In [129]:
last_months_transactions=data.groupby(['CustomerID'])['second_half'].sum().reset_index(name='last_months_transactions')

In [130]:
features = pd.merge(features, last_months_transactions, on="CustomerID")

### TransactionAmount (INR) median

In [132]:
median_amount= data.groupby(['CustomerID'])["TransactionAmount (INR)"].median().reset_index(name='median_amount')

In [134]:
features = pd.merge(features, median_amount, on="CustomerID")

### TransactionAmount (INR) first quartile

In [136]:
first_quartile_amount= data.groupby(['CustomerID'])["TransactionAmount (INR)"].quantile([0.25]).reset_index(name='first_quartile_amount')

In [138]:
features = pd.merge(features, first_quartile_amount, on="CustomerID")

### TransactionAmount (INR) third quartile

In [139]:
third_quartile_amount= data.groupby(['CustomerID'])["TransactionAmount (INR)"].quantile([0.75]).reset_index(name='third_quartile_amount')

In [140]:
features = pd.merge(features, third_quartile_amount, on="CustomerID")

In [143]:
features=features.drop(['level_1_x','level_1_y'], axis=1)

In [144]:
features

,CustomerID,n_transactions,n_transactions_100,average_amount,average_balance,utilisation,common_gender,frequent_location,min_trans,max_trans,...,percent_availability,earliest_TransactionTime,latest_TransactionTime,first_half_transactions,second_half_transactions,first_months_transactions,last_months_transactions,median_amount,first_quartile_amount,third_quartile_amount
0,C1010011,2,2,2553.0,76340.635,73787.635,"[F, M]","[NEW DELHI, NOIDA]",356.0,4750.0,...,96.66,01:12:29,12:38:13,1,1,0,2,2553.0,1454.50,3651.50
1,C1010012,1,1,1499.0,24204.490,22705.490,M,MUMBAI,1499.0,1499.0,...,93.81,20:44:09,20:44:09,0,1,0,1,1499.0,1499.00,1499.00
2,C1010014,2,2,727.5,100112.950,99385.450,"[F, M]",MUMBAI,250.0,1205.0,...,99.27,15:44:51,22:03:05,0,2,1,1,727.5,488.75,966.25
3,C1010018,1,0,30.0,496.180,466.180,F,CHAMPARAN,30.0,30.0,...,93.95,17:02:54,17:02:54,0,1,0,1,30.0,30.00,30.00
4,C1010024,1,1,5000.0,87058.650,82058.650,M,KOLKATA,5000.0,5000.0,...,94.26,14:11:03,14:11:03,0,1,0,1,5000.0,5000.00,5000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884260,C9099836,1,1,691.0,133067.230,132376.230,M,BHIWANDI,691.0,691.0,...,99.48,00:51:22,00:51:22,1,0,0,1,691.0,691.00,691.00
884261,C9099877,1,1,222.0,96063.460,95841.460,M,BANGALORE,222.0,222.0,...,99.77,12:02:55,12:02:55,0,1,0,1,222.0,222.00,222.00
884262,C9099919,1,1,126.0,5559.750,5433.750,M,GUNTUR,126.0,126.0,...,97.73,12:25:33,12:25:33,0,1,0,1,126.0,126.00,126.00
884263,C9099941,1,0,50.0,35295.920,35245.920,M,CHENNAI,50.0,50.0,...,99.86,21:37:22,21:37:22,0,1,0,1,50.0,50.00,50.00


In [146]:
features.to_csv('features.csv')